In [30]:
#%load_ext tensorboard
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorboard.plugins import projector
import json
from collections import defaultdict
import copy

from model import EmbeddingModel

In [33]:
data_dir = './data'
result_dir = './results'
log_dir = './logs/embedding/'
train_path = os.path.join(data_dir, 'train.zip')
hero_path = os.path.join(data_dir, 'hero_names.json')

embedding_size=16
dropout_rate=0
activation='tanh'
n_hidden_predictor=128
learning_rate=1e-3
batch_size=8196
epochs=150

In [34]:
with open(hero_path, 'r') as file:
    hero_names = json.load(file)

labeled_data = pd.read_csv(train_path)
no_winner = labeled_data['radiant_win'].isna()
labeled_data = labeled_data[~no_winner]

/tmp/ipykernel_2171/1148308636.py:4: DtypeWarning: Columns (102) have mixed types. Specify dtype option on import or set low_memory=False.
  labeled_data = pd.read_csv(train_path)


In [35]:
# Convert hero names json to be keyed on id
hero_id_info = {}
for name, hero_dict in hero_names.items():
    this_id = hero_dict['id']
    hero_id_info[this_id] = hero_dict

In [5]:
# Create train test/splits from match data
n_data = len(labeled_data)

random = np.random.RandomState(seed=116)
shuffled_rows = np.arange(n_data)
random.shuffle(shuffled_rows, )

train_frac = 0.8
validate_frac = 0.1
n_train = int(train_frac * n_data)
n_val = int(validate_frac * n_data)
train_rows = shuffled_rows[:n_train]
val_rows = shuffled_rows[n_train:n_train+n_val]
test_rows = shuffled_rows[n_train+n_val:]
n_test = len(test_rows)

train_data = labeled_data.iloc[train_rows, :]
val_data = labeled_data.iloc[val_rows, :]
test_data = labeled_data.iloc[test_rows, :]

In [6]:
def get_win_rates(train_radiant, train_dire, train_y, n_train):
    # Calculate historical winrates:
    win_counts = defaultdict(lambda: 0)
    game_counts = defaultdict(lambda: 0)
    for row in range(n_train):
        radiant = train_radiant.iloc[row, :]
        dire = train_dire.iloc[row, :]
        radiant_win = train_y.iloc[row]

        for hero in radiant:
            game_counts[hero] += 1
        for hero in dire:
            game_counts[hero] += 1

        if radiant_win:
            team = radiant
        else:
            team = dire

        for hero in team:
            win_counts[hero] += 1

    win_rates = {}
    for hero in win_counts.keys():
        win_rates[hero] = win_counts[hero] / game_counts[hero]

    return win_rates


def get_heroes_and_winner(df):
    radiant_cols = [f'r{idx}_hero' for idx in range(1,6)]
    dire_cols = [f'd{idx}_hero' for idx in range(1,6)]


    # make id's start at 0
    radiant_heroes = df[radiant_cols] - 1
    dire_heroes = df[dire_cols] - 1
    winners = df['radiant_win']

    return radiant_heroes, dire_heroes, winners

In [7]:
train_radiant, train_dire, train_y = get_heroes_and_winner(train_data)
val_radiant, val_dire, val_y = get_heroes_and_winner(val_data)
test_radiant, test_dire, test_y = get_heroes_and_winner(test_data)

In [29]:
win_rates = get_win_rates(train_radiant, train_dire, train_y, n_train)

# Serialize data into file:
winrates_path = os.path.join(result_dir, 'winrates.json')
json.dump(win_rates, open( winrates_path, 'w' ) )

# Feature of average team winrate

radiant_avg_wr = np.zeros(n_train)
dire_avg_wr = np.zeros(n_train)
val_radiant_wr = np.zeros(n_val)
val_dire_wr = np.zeros(n_val)
test_radiant_wr = np.zeros(n_test)
test_dire_wr = np.zeros(n_test)

for row in range(n_train):
    radiant = train_radiant.iloc[row, :]
    dire = train_dire.iloc[row, :]

    radiant_winrates = [win_rates[hero] for hero in radiant]
    dire_winrates = [win_rates[hero] for hero in dire]

    radiant_avg = np.mean(radiant_winrates)
    dire_avg = np.mean(dire_winrates)

    radiant_avg_wr[row] = radiant_avg
    dire_avg_wr[row] = dire_avg

for row in range(n_val):
    radiant = val_radiant.iloc[row, :]
    dire = val_dire.iloc[row, :]

    radiant_winrates = [win_rates[hero] for hero in radiant]
    dire_winrates = [win_rates[hero] for hero in dire]

    radiant_avg = np.mean(radiant_winrates)
    dire_avg = np.mean(dire_winrates)

    val_radiant_wr[row] = radiant_avg
    val_dire_wr[row] = dire_avg
    
for row in range(n_test):
    radiant = test_radiant.iloc[row, :]
    dire = test_dire.iloc[row, :]

    radiant_winrates = [win_rates[hero] for hero in radiant]
    dire_winrates = [win_rates[hero] for hero in dire]

    radiant_avg = np.mean(radiant_winrates)
    dire_avg = np.mean(dire_winrates)

    test_radiant_wr[row] = radiant_avg
    test_dire_wr[row] = dire_avg

In [9]:
worst_hero = min(win_rates, key=win_rates.get)
best_hero = max(win_rates, key=win_rates.get)
print(f'Best hero is {hero_id_info[best_hero]["localized_name"]} with a winrate of {100*win_rates[best_hero]:.2f}%')
print(f'Worst hero is {hero_id_info[worst_hero]["localized_name"]} with a winrate of {100*win_rates[worst_hero]:.2f}%')

Best hero is Clinkz with a winrate of 59.35%
Worst hero is Night Stalker with a winrate of 39.86%


In [10]:
model = EmbeddingModel(pool_size=112, embedding_size=embedding_size, dropout_rate=dropout_rate,
                           n_hidden_predictor=n_hidden_predictor, activation=activation)
loss = tf.keras.losses.BinaryCrossentropy()

2022-05-02 21:21:43.922578: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

csv_logs = os.path.join(log_dir, 'metrics.csv')
callbacks = [tf.keras.callbacks.TensorBoard(log_dir=log_dir),
             tf.keras.callbacks.CSVLogger(csv_logs)]

2022-05-02 21:21:43.946981: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2022-05-02 21:21:43.947010: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2022-05-02 21:21:43.947051: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.


In [12]:
optimizer = tf.keras.optimizers.Adam(learning_rate)
model.compile(optimizer=optimizer, loss=loss, metrics=[tf.metrics.BinaryAccuracy()])

In [13]:
model.fit(x=[tf.cast(train_radiant, dtype=tf.int32), tf.cast(train_dire, dtype=tf.int32),
             tf.cast(radiant_avg_wr, dtype=tf.float32), tf.cast(dire_avg_wr, dtype=tf.float32)],
          y=tf.cast(train_y.astype(int), dtype=tf.float32),
          callbacks=callbacks, shuffle=True,
          batch_size=batch_size, epochs=150,
          validation_data=
          ([tf.cast(val_radiant, dtype=tf.int32), tf.cast(val_dire, dtype=tf.int32),
            tf.cast(val_radiant_wr, dtype=tf.float32), tf.cast(val_dire_wr, dtype=tf.float32)],
           tf.cast(val_y.astype(int), dtype=tf.float32)))

Epoch 1/150


2022-05-02 21:21:44.015275: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-05-02 21:21:44.015642: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3700035000 Hz


16/40 [===========>..................] - ETA: 0s - loss: 0.6892 - binary_accuracy: 0.5378

2022-05-02 21:21:44.340671: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2022-05-02 21:21:44.340701: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2022-05-02 21:21:44.354132: I tensorflow/core/profiler/lib/profiler_session.cc:71] Profiler session collecting data.
2022-05-02 21:21:44.355171: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2022-05-02 21:21:44.360483: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: ./logs/embedding/train/plugins/profile/2022_05_02_21_21_44
2022-05-02 21:21:44.363382: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to ./logs/embedding/train/plugins/profile/2022_05_02_21_21_44/MrChipsNVME.trace.json.gz
2022-05-02 21:21:44.367424: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: ./logs/embedding/train/plugins/profile/2022_05_02_21_21

40/40 [==============================] - 1s 12ms/step - loss: 0.6826 - binary_accuracy: 0.5564 - val_loss: 0.6655 - val_binary_accuracy: 0.5955
Epoch 2/150
40/40 [==============================] - 0s 8ms/step - loss: 0.6636 - binary_accuracy: 0.5986 - val_loss: 0.6656 - val_binary_accuracy: 0.5947
Epoch 3/150
40/40 [==============================] - 0s 8ms/step - loss: 0.6635 - binary_accuracy: 0.5981 - val_loss: 0.6654 - val_binary_accuracy: 0.5945
Epoch 4/150
40/40 [==============================] - 0s 10ms/step - loss: 0.6632 - binary_accuracy: 0.5993 - val_loss: 0.6651 - val_binary_accuracy: 0.5941
Epoch 5/150
40/40 [==============================] - 0s 9ms/step - loss: 0.6629 - binary_accuracy: 0.5991 - val_loss: 0.6657 - val_binary_accuracy: 0.5955
Epoch 6/150
40/40 [==============================] - 0s 12ms/step - loss: 0.6642 - binary_accuracy: 0.5975 - val_loss: 0.6651 - val_binary_accuracy: 0.5950
Epoch 7/150
40/40 [==============================] - 0s 10ms/step - loss: 0.663

40/40 [==============================] - 0s 8ms/step - loss: 0.6634 - binary_accuracy: 0.5987 - val_loss: 0.6652 - val_binary_accuracy: 0.5961
Epoch 54/150
40/40 [==============================] - 0s 9ms/step - loss: 0.6627 - binary_accuracy: 0.5999 - val_loss: 0.6651 - val_binary_accuracy: 0.5954
Epoch 55/150
40/40 [==============================] - 0s 8ms/step - loss: 0.6634 - binary_accuracy: 0.5988 - val_loss: 0.6652 - val_binary_accuracy: 0.5948
Epoch 56/150
40/40 [==============================] - 0s 8ms/step - loss: 0.6628 - binary_accuracy: 0.5991 - val_loss: 0.6651 - val_binary_accuracy: 0.5960
Epoch 57/150
40/40 [==============================] - 0s 9ms/step - loss: 0.6636 - binary_accuracy: 0.5984 - val_loss: 0.6652 - val_binary_accuracy: 0.5961
Epoch 58/150
40/40 [==============================] - 0s 8ms/step - loss: 0.6630 - binary_accuracy: 0.5990 - val_loss: 0.6653 - val_binary_accuracy: 0.5960
Epoch 59/150
40/40 [==============================] - 0s 7ms/step - loss: 0.6

40/40 [==============================] - 0s 8ms/step - loss: 0.6597 - binary_accuracy: 0.6041 - val_loss: 0.6634 - val_binary_accuracy: 0.5972
Epoch 106/150
40/40 [==============================] - 0s 7ms/step - loss: 0.6599 - binary_accuracy: 0.6042 - val_loss: 0.6634 - val_binary_accuracy: 0.5991
Epoch 107/150
40/40 [==============================] - 0s 8ms/step - loss: 0.6600 - binary_accuracy: 0.6030 - val_loss: 0.6632 - val_binary_accuracy: 0.5981
Epoch 108/150
40/40 [==============================] - 0s 8ms/step - loss: 0.6596 - binary_accuracy: 0.6034 - val_loss: 0.6631 - val_binary_accuracy: 0.5992
Epoch 109/150
40/40 [==============================] - 0s 7ms/step - loss: 0.6593 - binary_accuracy: 0.6048 - val_loss: 0.6630 - val_binary_accuracy: 0.5986
Epoch 110/150
40/40 [==============================] - 0s 9ms/step - loss: 0.6596 - binary_accuracy: 0.6046 - val_loss: 0.6630 - val_binary_accuracy: 0.5991
Epoch 111/150
40/40 [==============================] - 0s 7ms/step - los

In [14]:
checkpoint_path = os.path.join(result_dir,'final_model.ckpt')
model.save_weights(checkpoint_path)

# Save Labels separately on a line-by-line manner.
with open(os.path.join(log_dir, 'metadata.tsv'), "w") as f:
  for id in range(1,113):
    if id in hero_id_info.keys():
        hero_name = hero_id_info[id]['localized_name']
        f.write(f'{hero_name}\n')
    else:
        f.write('Unknown\n')


# Save the weights we want to analyze as a variable. Note that the first
# value represents any unknown word, which is not in the metadata, here
# we will remove this value.
weights = tf.Variable(model.embedding.get_weights()[0])
# Create a checkpoint from embedding, the filename and key are the
# name of the tensor.
checkpoint = tf.train.Checkpoint(embedding=weights)
checkpoint.save(os.path.join(log_dir, "embedding.ckpt"))

'./logs/embedding/embedding.ckpt-1'

In [15]:
test_preds = model.call([tf.cast(test_radiant, dtype=tf.int32), tf.cast(test_dire, dtype=tf.int32),
             tf.expand_dims(tf.cast(test_radiant_wr, dtype=tf.float32),-1),
                         tf.expand_dims(tf.cast(test_dire_wr, dtype=tf.float32),-1)])
test_radiant_wins = test_preds>0.5

In [16]:
correct_test_preds = np.squeeze(test_radiant_wins.numpy())==test_y
incorrect_test_preds = np.squeeze(test_radiant_wins.numpy())!=test_y
test_acc  = correct_test_preds.mean()
print(f'Test acc {test_acc*100:0.2f}%')


Test acc 60.29%


In [17]:
correct_test_radiant, correct_test_dire, correct_test_y = get_heroes_and_winner(test_data[correct_test_preds])

In [18]:
def could_we_fix(radiant_rows, dire_rows, outcome, check_heros, single=False):
    
    if not single:
        n = len(radiant_rows)
    else:
        n = 1
    
    for row in range(n):
        
        if not single:
            radiant = radiant_rows.iloc[row, :]
            dire = dire_rows.iloc[row, :]
            radiant_won = outcome.iloc[row]
        else:
            radiant = radiant_rows
            dire = dire_rows
            radiant_won = outcome
        
        if radiant_won:
            losing_team = copy.deepcopy(dire)
            winning_team = copy.deepcopy(radiant)
        else:
            losing_team = copy.deepcopy(radiant)
            winning_team = copy.deepcopy(dire)
                
        for check_hero in check_heros:
            if (check_hero in radiant) or (check_hero in dire):
                continue
                
            
            # arbitrarily swap out first hero
            losing_team[0] = check_hero
            
            winning_winrates = [win_rates[hero] for hero in winning_team]
            losing_winrates = [win_rates[hero] for hero in losing_team]
            winning_avg = np.mean(winning_winrates)
            losing_avg = np.mean(losing_winrates)
            
            if radiant_won:
                new_preds = model.call([tf.expand_dims(tf.cast(winning_team, dtype=tf.int32), 0),
                                        tf.expand_dims(tf.cast(losing_team, dtype=tf.int32), 0),
                                        tf.expand_dims(tf.expand_dims(tf.cast(winning_avg, dtype=tf.float32), -1), -1),
                                        tf.expand_dims(tf.expand_dims(tf.cast(losing_avg, dtype=tf.float32), -1), -1)])
                if new_preds < 0.5:
                    print('Dire prediction flipped!')
                    print(dire)
            else:
                new_preds = model.call([tf.expand_dims(tf.cast(losing_team, dtype=tf.int32), 0),
                                        tf.expand_dims(tf.cast(winning_team, dtype=tf.int32), 0),
                                        tf.expand_dims(tf.expand_dims(tf.cast(losing_avg, dtype=tf.float32), -1), -1),
                                        tf.expand_dims(tf.expand_dims(tf.cast(winning_avg, dtype=tf.float32), -1), -1)])
                if new_preds > 0.5:
                    print('Radiant prediction flipped!')
                    print(new_preds.numpy())
                    print(radiant)
                
                
            

In [19]:
could_we_fix(correct_test_radiant, correct_test_dire, correct_test_y, [56])

Radiant prediction flipped!
[[0.54683626]]
r1_hero     10
r2_hero    111
r3_hero    109
r4_hero      6
r5_hero     71
Name: 370929, dtype: int64
Dire prediction flipped!
d1_hero     73
d2_hero    111
d3_hero     74
d4_hero     80
d5_hero     91
Name: 261993, dtype: int64
Dire prediction flipped!
d1_hero    99
d2_hero    25
d3_hero    72
d4_hero    71
d5_hero    92
Name: 351337, dtype: int64
Radiant prediction flipped!
[[0.60102224]]
r1_hero      7
r2_hero    103
r3_hero     34
r4_hero     30
r5_hero     68
Name: 59807, dtype: int64
Radiant prediction flipped!
[[0.5547871]]
r1_hero     72
r2_hero      2
r3_hero     50
r4_hero    105
r5_hero      8
Name: 220475, dtype: int64
Radiant prediction flipped!
[[0.5693077]]
r1_hero     85
r2_hero      7
r3_hero     41
r4_hero    102
r5_hero      1
Name: 341374, dtype: int64
Dire prediction flipped!
d1_hero     10
d2_hero     33
d3_hero    100
d4_hero     79
d5_hero    101
Name: 355039, dtype: int64
Dire prediction flipped!
d1_hero    105
d2_hero

Dire prediction flipped!
d1_hero     14
d2_hero     26
d3_hero      6
d4_hero     25
d5_hero    105
Name: 266607, dtype: int64
Radiant prediction flipped!
[[0.6257628]]
r1_hero    10
r2_hero    92
r3_hero    82
r4_hero     7
r5_hero    67
Name: 33148, dtype: int64
Dire prediction flipped!
d1_hero    73
d2_hero    71
d3_hero    22
d4_hero    10
d5_hero    61
Name: 334419, dtype: int64
Radiant prediction flipped!
[[0.540299]]
r1_hero    97
r2_hero     0
r3_hero    25
r4_hero    34
r5_hero    43
Name: 157906, dtype: int64
Dire prediction flipped!
d1_hero    105
d2_hero     54
d3_hero     68
d4_hero     14
d5_hero      6
Name: 938, dtype: int64
Dire prediction flipped!
d1_hero     15
d2_hero      0
d3_hero     20
d4_hero    100
d5_hero     72
Name: 287601, dtype: int64
Dire prediction flipped!
d1_hero     38
d2_hero    103
d3_hero      0
d4_hero     56
d5_hero     25
Name: 224645, dtype: int64
Dire prediction flipped!
d1_hero     97
d2_hero     10
d3_hero     67
d4_hero     72
d5_hero    1

Radiant prediction flipped!
[[0.53643215]]
r1_hero    111
r2_hero     45
r3_hero     27
r4_hero     61
r5_hero     38
Name: 258525, dtype: int64
Dire prediction flipped!
d1_hero    105
d2_hero     75
d3_hero    103
d4_hero      8
d5_hero     29
Name: 28892, dtype: int64
Dire prediction flipped!
d1_hero    92
d2_hero    49
d3_hero    30
d4_hero    38
d5_hero    68
Name: 349966, dtype: int64
Radiant prediction flipped!
[[0.5376551]]
r1_hero    16
r2_hero    64
r3_hero    49
r4_hero    35
r5_hero     0
Name: 169651, dtype: int64
Dire prediction flipped!
d1_hero    104
d2_hero     20
d3_hero     76
d4_hero     48
d5_hero     10
Name: 159084, dtype: int64
Radiant prediction flipped!
[[0.5674968]]
r1_hero     15
r2_hero    102
r3_hero     27
r4_hero      4
r5_hero     33
Name: 77735, dtype: int64
Dire prediction flipped!
d1_hero      3
d2_hero    111
d3_hero      0
d4_hero     85
d5_hero     31
Name: 322429, dtype: int64
Dire prediction flipped!
d1_hero     50
d2_hero    110
d3_hero     92
d

Radiant prediction flipped!
[[0.6115719]]
r1_hero     20
r2_hero     73
r3_hero     71
r4_hero     86
r5_hero    101
Name: 25912, dtype: int64
Radiant prediction flipped!
[[0.5973574]]
r1_hero    96
r2_hero     9
r3_hero    67
r4_hero    31
r5_hero    21
Name: 27030, dtype: int64
Dire prediction flipped!
d1_hero    92
d2_hero    84
d3_hero    98
d4_hero    48
d5_hero    90
Name: 110721, dtype: int64
Radiant prediction flipped!
[[0.53484064]]
r1_hero     24
r2_hero    100
r3_hero     83
r4_hero     92
r5_hero      6
Name: 235129, dtype: int64
Dire prediction flipped!
d1_hero    60
d2_hero     5
d3_hero    20
d4_hero    74
d5_hero    41
Name: 374000, dtype: int64
Dire prediction flipped!
d1_hero     4
d2_hero    40
d3_hero    73
d4_hero    29
d5_hero    68
Name: 281821, dtype: int64
Dire prediction flipped!
d1_hero     0
d2_hero    27
d3_hero    17
d4_hero     2
d5_hero    22
Name: 196167, dtype: int64
Radiant prediction flipped!
[[0.5183708]]
r1_hero    71
r2_hero    14
r3_hero    67
r4

Dire prediction flipped!
d1_hero    16
d2_hero    39
d3_hero    34
d4_hero    83
d5_hero    56
Name: 395263, dtype: int64
Radiant prediction flipped!
[[0.54669094]]
r1_hero    24
r2_hero    72
r3_hero    25
r4_hero    13
r5_hero    75
Name: 164142, dtype: int64
Dire prediction flipped!
d1_hero    98
d2_hero    13
d3_hero    58
d4_hero    99
d5_hero    48
Name: 64615, dtype: int64
Radiant prediction flipped!
[[0.5422837]]
r1_hero     61
r2_hero     10
r3_hero     59
r4_hero    111
r5_hero     18
Name: 263759, dtype: int64
Dire prediction flipped!
d1_hero    73
d2_hero    25
d3_hero    92
d4_hero    18
d5_hero    72
Name: 203621, dtype: int64
Radiant prediction flipped!
[[0.6059964]]
r1_hero      2
r2_hero     92
r3_hero     46
r4_hero     37
r5_hero    100
Name: 323056, dtype: int64
Dire prediction flipped!
d1_hero    40
d2_hero    10
d3_hero    67
d4_hero    13
d5_hero    38
Name: 196374, dtype: int64
Dire prediction flipped!
d1_hero    67
d2_hero    96
d3_hero    92
d4_hero    89
d5_h

...

Radiant prediction flipped!
[[0.50671726]]
r1_hero     13
r2_hero     10
r3_hero    105
r4_hero     92
r5_hero     83
Name: 95362, dtype: int64
Dire prediction flipped!
d1_hero     43
d2_hero     20
d3_hero     76
d4_hero     46
d5_hero    110
Name: 12985, dtype: int64
Dire prediction flipped!
d1_hero     8
d2_hero    43
d3_hero    84
d4_hero    33
d5_hero    51
Name: 139965, dtype: int64
Dire prediction flipped!
d1_hero    24
d2_hero    84
d3_hero    66
d4_hero     9
d5_hero    70
Name: 293702, dtype: int64
Dire prediction flipped!
d1_hero     6
d2_hero    28
d3_hero    19
d4_hero    16
d5_hero     7
Name: 148830, dtype: int64
Radiant prediction flipped!
[[0.6379345]]
r1_hero     9
r2_hero    29
r3_hero    10
r4_hero    96
r5_hero    27
Name: 287049, dtype: int64
Radiant prediction flipped!
[[0.6475424]]
r1_hero     2
r2_hero    97
r3_hero    63
r4_hero    20
r5_hero    92
Name: 17800, dtype: int64
Dire prediction flipped!
d1_hero     10
d2_hero     38
d3_hero     92
d4_hero    101
d5

Radiant prediction flipped!
[[0.68238515]]
r1_hero    64
r2_hero    80
r3_hero    82
r4_hero    92
r5_hero    31
Name: 387358, dtype: int64
Dire prediction flipped!
d1_hero      3
d2_hero     43
d3_hero    105
d4_hero     27
d5_hero     73
Name: 135226, dtype: int64
Radiant prediction flipped!
[[0.55682385]]
r1_hero     67
r2_hero     10
r3_hero    103
r4_hero     27
r5_hero     22
Name: 54368, dtype: int64
Dire prediction flipped!
d1_hero    105
d2_hero     52
d3_hero    101
d4_hero      5
d5_hero     10
Name: 176431, dtype: int64
Radiant prediction flipped!
[[0.54000425]]
r1_hero     73
r2_hero     48
r3_hero     49
r4_hero      1
r5_hero    105
Name: 71100, dtype: int64
Dire prediction flipped!
d1_hero    26
d2_hero    73
d3_hero    52
d4_hero    61
d5_hero    92
Name: 182107, dtype: int64
Dire prediction flipped!
d1_hero    38
d2_hero    99
d3_hero    49
d4_hero    54
d5_hero    71
Name: 164527, dtype: int64
Radiant prediction flipped!
[[0.5055448]]
r1_hero     27
r2_hero     20
r3

Dire prediction flipped!
d1_hero     30
d2_hero      0
d3_hero    111
d4_hero     38
d5_hero     99
Name: 350323, dtype: int64
Dire prediction flipped!
d1_hero     18
d2_hero     59
d3_hero     54
d4_hero     98
d5_hero    105
Name: 63275, dtype: int64
Radiant prediction flipped!
[[0.5063546]]
r1_hero     38
r2_hero     27
r3_hero     20
r4_hero     78
r5_hero    105
Name: 345613, dtype: int64
Radiant prediction flipped!
[[0.5513774]]
r1_hero    90
r2_hero    34
r3_hero    11
r4_hero    40
r5_hero    22
Name: 1984, dtype: int64
Radiant prediction flipped!
[[0.6062838]]
r1_hero    62
r2_hero    18
r3_hero    85
r4_hero    20
r5_hero    82
Name: 291933, dtype: int64
Dire prediction flipped!
d1_hero     6
d2_hero    53
d3_hero    73
d4_hero    34
d5_hero    29
Name: 85393, dtype: int64
Dire prediction flipped!
d1_hero    22
d2_hero    61
d3_hero    34
d4_hero     6
d5_hero    13
Name: 119132, dtype: int64
Dire prediction flipped!
d1_hero      0
d2_hero    101
d3_hero     59
d4_hero     61

Radiant prediction flipped!
[[0.6331071]]
r1_hero    103
r2_hero     52
r3_hero     35
r4_hero     20
r5_hero      7
Name: 103418, dtype: int64
Radiant prediction flipped!
[[0.53162557]]
r1_hero     2
r2_hero    17
r3_hero    51
r4_hero    20
r5_hero    97
Name: 51765, dtype: int64
Dire prediction flipped!
d1_hero    47
d2_hero    20
d3_hero    76
d4_hero    44
d5_hero    10
Name: 113194, dtype: int64
Dire prediction flipped!
d1_hero     63
d2_hero    103
d3_hero    105
d4_hero     21
d5_hero     45
Name: 400447, dtype: int64
Dire prediction flipped!
d1_hero     4
d2_hero    72
d3_hero    99
d4_hero    20
d5_hero    46
Name: 363721, dtype: int64
Radiant prediction flipped!
[[0.51503927]]
r1_hero    72
r2_hero    14
r3_hero    86
r4_hero    84
r5_hero    27
Name: 6292, dtype: int64
Dire prediction flipped!
d1_hero    99
d2_hero    10
d3_hero    69
d4_hero    73
d5_hero    13
Name: 254305, dtype: int64
Dire prediction flipped!
d1_hero    40
d2_hero    17
d3_hero    67
d4_hero    10
d5_he

Radiant prediction flipped!
[[0.5931599]]
r1_hero     19
r2_hero    111
r3_hero     68
r4_hero     12
r5_hero      0
Name: 21007, dtype: int64
Radiant prediction flipped!
[[0.5885006]]
r1_hero    12
r2_hero    64
r3_hero    38
r4_hero    70
r5_hero    29
Name: 11203, dtype: int64
Radiant prediction flipped!
[[0.6934315]]
r1_hero    38
r2_hero    24
r3_hero    19
r4_hero    83
r5_hero    80
Name: 233406, dtype: int64
Dire prediction flipped!
d1_hero    101
d2_hero     98
d3_hero     10
d4_hero    111
d5_hero     93
Name: 54141, dtype: int64
Dire prediction flipped!
d1_hero    13
d2_hero     2
d3_hero    16
d4_hero    61
d5_hero    55
Name: 226298, dtype: int64
Dire prediction flipped!
d1_hero      9
d2_hero     26
d3_hero     84
d4_hero     24
d5_hero    103
Name: 320387, dtype: int64


In [20]:
test_data.loc[341374]

match_id                    1993548568
r1_hero                             86
r1_level                             0
r1_xp                              917
r1_gold                            589
                               ...    
dire_ward_observer_count             2
dire_ward_sentry_count               0
dire_first_ward_time             -20.0
duration                          2420
radiant_win                      False
Name: 341374, Length: 103, dtype: object

In [21]:
hero_id_info

{1: {'id': 1,
  'name': 'npc_dota_hero_antimage',
  'localized_name': 'Anti-Mage',
  'primary_attr': 'agi',
  'attack_type': 'Melee',
  'roles': ['Carry', 'Escape', 'Nuker'],
  'img': '/apps/dota2/images/heroes/antimage_full.png?',
  'icon': '/apps/dota2/images/heroes/antimage_icon.png',
  'base_health': 200,
  'base_health_regen': 0.25,
  'base_mana': 75,
  'base_mana_regen': 0,
  'base_armor': -1,
  'base_mr': 25,
  'base_attack_min': 29,
  'base_attack_max': 33,
  'base_str': 23,
  'base_agi': 24,
  'base_int': 12,
  'str_gain': 1.3,
  'agi_gain': 3.2,
  'int_gain': 1.8,
  'attack_range': 150,
  'projectile_speed': 0,
  'attack_rate': 1.4,
  'move_speed': 310,
  'turn_rate': 0.5,
  'cm_enabled': True,
  'legs': 2},
 2: {'id': 2,
  'name': 'npc_dota_hero_axe',
  'localized_name': 'Axe',
  'primary_attr': 'str',
  'attack_type': 'Melee',
  'roles': ['Initiator', 'Durable', 'Disabler', 'Jungler'],
  'img': '/apps/dota2/images/heroes/axe_full.png?',
  'icon': '/apps/dota2/images/heroes/

In [22]:
could_we_fix(correct_test_radiant.loc[341374], correct_test_dire.loc[341374], correct_test_y.loc[341374], [49], single=True)

Radiant prediction flipped!
[[0.5311771]]
r1_hero     85
r2_hero      7
r3_hero     41
r4_hero    102
r5_hero      1
Name: 341374, dtype: int64


In [23]:
pd.Series(correct_test_radiant.loc[341374])

r1_hero     85
r2_hero      7
r3_hero     41
r4_hero    102
r5_hero      1
Name: 341374, dtype: int64

In [24]:
test_radiant_wins

<tf.Tensor: shape=(40980, 1), dtype=bool, numpy=
array([[ True],
       [ True],
       [ True],
       ...,
       [ True],
       [ True],
       [False]])>

In [25]:
np.where(correct_test_radiant.index==341374)

(array([10]),)

In [26]:
test_preds.numpy()[10]

array([0.4497685], dtype=float32)

In [27]:
win_rates

{96: 0.4551410757687837,
 24: 0.44779573029638425,
 4: 0.5396770272616773,
 20: 0.48965437854326743,
 10: 0.5232406126246482,
 15: 0.46896851039205284,
 13: 0.487533300365307,
 21: 0.5566216169744489,
 29: 0.5210636055870681,
 80: 0.5147838809973785,
 103: 0.4847440446021287,
 84: 0.5556154398932348,
 105: 0.46499661101351447,
 35: 0.5681681337270981,
 58: 0.501174389165116,
 92: 0.49294681761206505,
 74: 0.5242131350681537,
 7: 0.5308666657146529,
 72: 0.5081051362088536,
 99: 0.49733225076842774,
 95: 0.51657229664411,
 73: 0.49533349074775995,
 66: 0.5804479381121717,
 8: 0.5200263923169033,
 19: 0.5268433799784715,
 40: 0.4479508581074744,
 43: 0.47828277297119703,
 69: 0.5654082528533801,
 106: 0.5040004706436051,
 14: 0.48405939897248224,
 25: 0.47113666006473115,
 71: 0.5008884073672806,
 38: 0.4433722021176786,
 2: 0.47351690999815194,
 11: 0.46600799811808985,
 83: 0.5111085307323116,
 89: 0.483013589128697,
 86: 0.5138507774728864,
 0: 0.47466951788491446,
 52: 0.423126119212